In [1]:
!pip install transformers

     |████████████████████████████████| 1.1MB 5.2MB/s 
     |████████████████████████████████| 890kB 29.1MB/s 
     |████████████████████████████████| 3.0MB 40.5MB/s 
     |████████████████████████████████| 1.1MB 39.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=1597768b9f8e01b3b4bbc1a808af6872085a51ea567aedcde806c86908131fc9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
import random
from transformers import BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
import time
import datetime
#import logging
#import matplotlib.pyplot as plt
#import seaborn as sns
#from sklearn.model_selection import train_test_split
# keras.preprocessing.sequence import pad_sequences
#from torch.utilis.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
#import time

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("there are %d GPU(s) available." % torch.cuda.device_count())
  print("The GPU used in this code is: ", torch.cuda.get_device_name(0))
else:
  assert False, "No GPU detected. Please, select GPU in the Colab."

there are 1 GPU(s) available.
The GPU used in this code is:  Tesla T4


In [5]:
df = pd.read_csv("/content/drive/My Drive/Disaster-Or-Not/data/train.csv", delimiter=",", header=0, names=["id", "keyword", "location", "text","target"])

In [6]:
df.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
print("dimensions of training data:", df.shape)

dimensions of training data: (7613, 5)


In [8]:
print("the number of sentences seen as disaster telling:", df[df.target==1].shape[0])

the number of sentences seen as disaster telling: 3271


In [9]:
print("the number of observations of keyword == naN is:",df[pd.isna(df.keyword)].shape[0])

the number of observations of keyword == naN is: 61


In [10]:
df[df.target == 1].sample(5)[['text']]

,text
7041,Obama Declares Disaster for Typhoon-Devastated...
7253,'The Reagan Administration had arranged for Is...
7435,Gunshot wound #9 is in the bicep. The only one...
1187,Ashes 2015: Australia collapse at Trent Bridge...
4658,This is set to become a huge one month wonder....


In [11]:
texts = df.text.values
targets = df.target.values

In [12]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [13]:
print(texts[150])
print(tokenizer.tokenize(texts[150]))

@mickinyman @TheAtlantic That or they might be killed in an airplane accident in the night a car wreck! Politics at it's best.
['@', 'mick', '##iny', '##man', '@', 'the', '##at', '##lan', '##tic', 'that', 'or', 'they', 'might', 'be', 'killed', 'in', 'an', 'airplane', 'accident', 'in', 'the', 'night', 'a', 'car', 'wreck', '!', 'politics', 'at', 'it', "'", 's', 'best', '.']


In [14]:
max_len = 0

for text in texts:
    input_ids = tokenizer.encode(text, add_special_tokens=True)
    max_len = max(max_len,len(input_ids))
print("max length to set to is", max_len)

max length to set to is 84


In [15]:
input_ids = []
attention_masks = []

for text in texts:
    encoded_dict = tokenizer.encode_plus(text, add_special_tokens=True, max_length=84, truncation=True, pad_to_max_length=True, return_attention_mask=True, return_tensors='pt')
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids,dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
targets = torch.tensor(targets)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [16]:
dataset = TensorDataset(input_ids, attention_masks, targets)

train_size = int(0.9 * len(dataset))
val_size = len(dataset)-train_size

random.seed(10)
train_dataset, val_dataset = random_split(dataset,[train_size,val_size])

In [17]:

batch_size = 32

train_data_loader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size= batch_size)
validation_data_loader = DataLoader(val_dataset, sampler = SequentialSampler(val_dataset), batch_size = batch_size)

In [18]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2, output_attentions = False, output_hidden_states = False)

model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [19]:
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)
epochs = 4

total_steps = len(train_data_loader)*epochs

scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0, num_training_steps = total_steps)

In [20]:
def flat_accuracy (preds, labels):
    pred_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat)/len(labels_flat)

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round(elapsed))))

In [21]:


random.seed(32)
np.random.seed(32)
torch.manual_seed(32)


#Storing training stats
training_stats = []

total_t0 = time.time()


for epoch_i in range(0, epochs):
    print("")
    print("================ Epoch {:} / {:} ================".format(epoch_i + 1, epochs))
    print('Training ongoing...')

    t0 = time.time()

    #reseting total loss for current epoch
    total_train_loss = 0

    #Activating train mode
    model.train

    for step, batch in enumerate(train_data_loader):
        
        # Progress visualization every 15 batches
        if step % 15 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print(' Batch {: >5,} of {:>5,}.  Elapsed: {:}.'.format(step, len(train_data_loader), elapsed))
        
        # Unpacking batches
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        #Clear Grads
        model.zero_grad()

        #performing forward pass
        loss, loggits = model(b_input_ids, token_type_ids = None, attention_mask = b_input_mask, labels= b_labels)

        #Accumulating training loss
        total_train_loss += loss.item()

        # Performing Backward pass
        loss.backward()

        # Setting the norm of gradients to 1.0.
        torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)

        #Updating parameters
        optimizer.step()

        #Updating the learning rate
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_data_loader)
    training_time = format_time(time.time() - t0)

    print("")
    print(" Avg training loss: {0:.2f}".format(avg_train_loss))
    print(" Training epoch took: {:}".format(training_time)) 

    print("Running validation")

    t0 = time.time()

    model.eval()


    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_data_loader:

      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)

      with torch.no_grad():

        (loss, logits) = model(b_input_ids,token_type_ids = None, attention_mask = b_input_mask, labels = b_labels)

      total_eval_loss += loss.item()

      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()

      total_eval_accuracy += flat_accuracy(logits, label_ids)

    print("validation Over")

    avg_val_accuracy = total_eval_accuracy/len(validation_data_loader)
    avg_val_loss = total_eval_loss/len(validation_data_loader)
    validation_time = format_time(time.time()-t0)

    print(" Accuracy: {0:.2f}".format(avg_val_accuracy))
    print(" Validation Loss {0:.2f}".format(avg_val_loss))
    print(" Validation took: {:}".format(validation_time))

    training_stats.append({"epoch": epoch_i + 1,
                         "training_loss": avg_train_loss,
                         "val_loss": avg_val_loss,
                         "val_accuracy": avg_val_accuracy,
                         "training_time": training_time,
                         "validation_time": validation_time
                         })

print("Training Over!!!!")
print("it took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))



================ Epoch 1 / 4 ================
Training ongoing...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


 Batch    15 of   215.  Elapsed: 0:00:07.
 Batch    30 of   215.  Elapsed: 0:00:13.
 Batch    45 of   215.  Elapsed: 0:00:20.
 Batch    60 of   215.  Elapsed: 0:00:26.
 Batch    75 of   215.  Elapsed: 0:00:33.
 Batch    90 of   215.  Elapsed: 0:00:40.
 Batch   105 of   215.  Elapsed: 0:00:46.
 Batch   120 of   215.  Elapsed: 0:00:53.
 Batch   135 of   215.  Elapsed: 0:01:00.
 Batch   150 of   215.  Elapsed: 0:01:07.
 Batch   165 of   215.  Elapsed: 0:01:14.
 Batch   180 of   215.  Elapsed: 0:01:21.
 Batch   195 of   215.  Elapsed: 0:01:28.
 Batch   210 of   215.  Elapsed: 0:01:36.

 Avg training loss: 0.44
 Training epoch took: 0:01:38
Running validation
validation Over
 Accuracy: 0.84
 Validation Loss 0.39
 Validation took: 0:00:04

================ Epoch 2 / 4 ================
Training ongoing...
 Batch    15 of   215.  Elapsed: 0:00:07.
 Batch    30 of   215.  Elapsed: 0:00:14.
 Batch    45 of   215.  Elapsed: 0:00:21.
 Batch    60 of   215.  Elapsed: 0:00:28.
 Batch    75 of   215.

Dealing with test data set now

In [22]:
testdata = pd.read_csv("/content/drive/My Drive/Disaster-Or-Not/data/test.csv", delimiter=",", header=0, names=["id", "keyword", "location", "text"])

In [23]:
test_texts = testdata.text.values

In [24]:
test_input_ids = []
test_attention_masks = []

for text in test_texts: 
  test_encoded_dict = tokenizer.encode_plus(text, add_special_tokens=True, max_length=84, truncation=True, pad_to_max_length = True, return_attention_mask = True, return_tensors='pt')
  test_input_ids.append(test_encoded_dict['input_ids'])
  test_attention_masks.append(test_encoded_dict['attention_mask'])

test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)
print("yo")

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


yo


In [28]:
test_real_data = TensorDataset(test_input_ids, test_attention_masks)

In [32]:
test_data_loader = DataLoader(test_real_data, sampler = SequentialSampler(testdata), batch_size= batch_size)

In [49]:
model.eval()

preds = []
for step, batch in enumerate(test_data_loader):

  if step % 6 == 0 and not step == 0:
    print(f"==============================Batch {step} out of {len(test_data_loader)} =========================")
    print("\n")

  b_test_input_ids = batch[0].to(device)
  b_test_input_masks = batch[1].to(device)

  with torch.no_grad():
    outputs = model(b_test_input_ids, token_type_ids=None, attention_mask=b_test_input_masks)
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    logits = np.argmax(logits, axis=1).flatten()
    preds.append(logits)

preds = [element for my_list in preds for element in my_list]

print("Outputs released. OVER!!")

==============================Batch 6 out of 102 =========================


==============================Batch 12 out of 102 =========================


==============================Batch 18 out of 102 =========================


==============================Batch 24 out of 102 =========================


==============================Batch 30 out of 102 =========================


==============================Batch 36 out of 102 =========================


==============================Batch 42 out of 102 =========================


==============================Batch 48 out of 102 =========================


==============================Batch 54 out of 102 =========================


==============================Batch 60 out of 102 =========================


==============================Batch 66 out of 102 =========================


==============================Batch 72 out of 102 =========================


==============================Batch 78 out of 102 ===============

In [1]:
output = pd.DataFrame( {'id': testdata.id, 'target': preds})
output.to_csv ('/content/drive/My Drive/Disaster-Or-Not/data/my_submission.csv', index=False)
print("Submission file successfully saved!")

NameError: ignored